In [24]:
import requests
from io import StringIO


In [5]:
import pandas as pd

In [6]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [10]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [12]:
sentence_b = data['sentence_B'].tolist()
sentence_b[:4]

['A group of boys in a yard is playing and a man is standing in the background',
 'A group of kids is playing in a yard and an old man is standing in the background',
 'The kids are playing outdoors near a man with a smile',
 'A group of kids is playing in a yard and an old man is standing in the background']

In [13]:
sentences.extend(sentence_b)

In [19]:
len(set(sentences))

4802

In [21]:
len(sentences)

9000

In [22]:
len(sentence_b)

4500

In [23]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [33]:
for url in urls:
    result = requests.get(url)
    data = pd.read_csv(StringIO(result.text), sep='\t', header=None, on_bad_lines='skip')
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [40]:
len(sentences)

20470

In [38]:
len(set(sentences))

14505

In [42]:
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [43]:
len(sentences)

14504

In [ ]:
pip install sentence-transformers

In [49]:
from sentence_transformers import SentenceTransformer

C:\Git\LLM\AI Agent\signeture-verification\signeture-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings_c = model.encode("The boys are good")

In [56]:
sentence_embeddings_c.shape

(768,)

In [58]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings_b = model.encode(sentence_b)

In [59]:
sentence_embeddings_b.shape

(4500, 768)

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(sentences)

In [62]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.11.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached faiss_cpu-1.11.0-cp312-cp312-win_amd64.whl (15.0 MB)


In [64]:
import faiss
print(faiss.__version__)

1.11.0


In [66]:
import faiss
d = sentence_embeddings_b.shape[1]
d

768

In [67]:
index = faiss.IndexFlatL2(d) #Euclidean Distance L2

In [68]:
index.is_trained

True

In [69]:
index.add(sentence_embeddings_b)

In [70]:
index.ntotal

4500

In [74]:
# Now we are searching the given query

In [108]:
limit_k = 4
query_vector = model.encode(["Someone sprints with a football"])

In [109]:
%%time
D, I = index.search(query_vector, limit_k)
print(I, D)

[[3200 3203 3204 3202]] [[54.62377  54.62377  54.62377  54.853477]]
CPU times: total: 0 ns
Wall time: 2.99 ms


In [119]:
#D.shape
I.shape
#D[0].shape
#I[0].shape

(1, 4)

In [87]:
len(sentence_b)

4500

In [98]:
sentence_b[3202]

'A group of people playing football is running in the field'

In [118]:
indices = I[0]
distances = D[0]

pd.DataFrame({
    "Distance, L2": distances,
    "Matched Sentence": [sentence_b[i] for i in indices]
})

,"Distance, L2",Matched Sentence
0,54.623772,A group of football players is running in the ...
1,54.623772,A group of football players is running in the ...
2,54.623772,A group of football players is running in the ...
3,54.853477,A group of people playing football is running ...
